In [1]:
import numpy as np
import pandas as pd

In [2]:
def read_input(path):
    """ Reads input file from passed path and returns as numpy array. No
    preprocessing is done. """
    with open(path, 'r') as f:
        data = np.array([l.strip() for l in f.readlines()])
    return data

### Day 1

#### Part 1

In [4]:
day_1_data = read_input("day_1/input.txt").astype(int)

In [5]:
def calc_positive_diff(arr):
    """ Given an array, count how many 'next values' are higher then the previous value. """
    return sum((arr[1:] - arr[:-1]) > 0)


def sum_by_window(arr, window_size=3):
    """ Slides a window_size sized window over an array and calculates the sums of the windows. """
    return pd.Series(arr).rolling(window=window_size).sum().dropna().values

In [6]:
calc_positive_diff(day_1_data)

1692

#### Part 2

In [7]:
# Calculate diff using sum of sliding window
calc_positive_diff(sum_by_window(day_1_data, window_size=3))

1724

### Day 2

#### Part 1

In [8]:
day_2_data = read_input("day_2/input.txt")

# Split lines into command and values and converts the values into integers
day_2_data = [[l[0], int(l[1])] for l in [line.split() for line in day_2_data]]

In [9]:
x_pos = 0
y_pos = 0

for command, value in day_2_data:
    if command.startswith('f'):
        x_pos += value
    elif command.startswith('d'):
        y_pos += value
    elif command.startswith('u'):
        y_pos -= value
        
x_pos * y_pos

1250395

#### Part 2

In [10]:
x_pos = 0
y_pos = 0
aim = 0

for command, value in day_2_data:
    if command.startswith('f'):
        x_pos += value
        y_pos += value * aim
    elif command.startswith('d'):
        aim += value
    elif command.startswith('u'):
        aim -= value
        
x_pos * y_pos

1451210346

In [11]:
# Numpy solution part 1
day_2_data = read_input("day_2/input.txt")
commands, values = zip(*[line.split() for line in day_2_data])
# Turn into arrays for slicing possibilities
commands = np.array(commands)
values = np.array(values).astype(int)

x_pos = values[commands == 'forward'].sum()
y_pos = values[commands == 'down'].sum() - values[commands == 'up'].sum()
x_pos * y_pos

1250395

### Day 3

In [24]:
def load_day_3_data():
    day_3_data = read_input("day_3/input.txt")

    # Turn into numpy array, every bit on its own
    day_3_data = np.array([[int(bit) for bit in bytes_] for bytes_ in day_3_data])
    
    return day_3_data

#### Part 1

In [25]:
day_3_data = load_day_3_data()

In [26]:
# Gamma is most commit bit per 'column'
gamma = np.array([np.argmax(np.bincount(day_3_data[:, i])) for i in range(day_3_data.shape[1])])

# Epsilon is least common bit per column, so the opposite of the gamma
epsilon = 1 - np.array(gamma)

In [27]:
def byte_array_to_int(arr):
    return int(''.join(arr.astype(str)), base=2)

In [28]:
# Transform binary to number
gamma = byte_array_to_int(gamma)
epsilon = byte_array_to_int(epsilon)

In [29]:
# Answer is gamma times epsilon
gamma * epsilon

1458194

#### Part 2

In [30]:
day_3_data = load_day_3_data()

In [31]:
def day_3_part_2(data, filter_by='max'):
    assert filter_by in ['min', 'max']
    
    if filter_by == 'min':
        default = 0
        min_max_func = np.argmin
    else:
        default = 1
        min_max_func = np.argmax
    
    for idx in range(data.shape[1]):
        count = np.bincount(data[:, idx])
        if len(set(count)) > 1 or len(set(data[:, idx])) == 1:
            mcv = min_max_func(count)
        # Value if column contains the same amount of both values
        else:
            mcv = default

        # Remove rows where value of current index is not the most common value
        data = data[data[:, idx] == mcv]
        if data.shape[0] <= 1:
            break
            
    return data

In [34]:
oxygen_rating = byte_array_to_int(day_3_part_2(day_3_data, filter_by='max')[0])
co2_rating = byte_array_to_int(day_3_part_2(day_3_data, filter_by='min')[0])

In [35]:
oxygen_rating * co2_rating

2829354

### Day 4

#### Part 1

In [141]:
with open('day_4/input.txt', 'r') as f:
    input_data = f.readlines()
    # Split comma separated string of numbers to list and parse all
    # string numbers to ints
    numbers = [int(i) for i in input_data[0].strip().split(',')]
    # The rest of the data are the bingo cards
    bingo_cards = [line.strip().split() for line in input_data[2:] if line != '\n']  # Remove empty lines
    
    # Using reshape get a 3d array of N 5 by 5 bingo cards
    bingo_card_numbers = np.array(bingo_cards).reshape(-1, 5, 5).astype(int)

In [152]:
class BingoCard:
    
    def __init__(self, values):
        self.values = values  # 2D array
        self.marked = np.zeros(shape=self.values.shape)  # Keep track of numbers that we marked
        self.card_size = self.values.shape[0]  # Assumes square card
        self.latest_value = None  # Needed for calculating the final answer
        
    def mark_number(self, value):
        self.latest_value = value
        # Mark a 1 on the indexes where we have a hit
        if value in self.values:
            self.marked[np.where(self.values==value)] = 1
            
    def check_for_bingo(self):
        # If the sum of a row/column is the same as the size of the card we have a bingo
        # Diagonals are not counted so this is easy
        return self.card_size in self.marked.sum(axis=0) or self.card_size in self.marked.sum(axis=1)
    
    def calculate_answer(self):
        # Sum of unmarked values * latest called value
        return self.values[self.marked == 0].sum() * self.latest_value
    
    def calculate_n_moves_until_win(self, bingo_numbers):
        """ Needed for part 2 """
        for n, number in enumerate(bingo_numbers):
            self.mark_number(number)
            if self.check_for_bingo():
                return n, self.calculate_answer()

In [146]:
bingo_cards = [BingoCard(card) for card in bingo_card_numbers]

for number in numbers:
    # Fill number on card
    # Dont assign to variable as we're updating class instances
    [card.mark_number(number) for card in bingo_cards]
    
    # Check for bingos
    bingo_checks = [card.check_for_bingo() for card in bingo_cards]
    
    # If we have bingo there will be a true in this list, we check this with any
    if any(bingo_checks):
        # Winning card is the index of True
        winning_card = bingo_cards[bingo_checks.index(True)]
        break
        
winning_card.calculate_answer()

38913

#### Part 2

In [148]:
bingo_cards = [BingoCard(card) for card in bingo_card_numbers]

# Get list of tuples containing amount of moves until win, and the score of the card
moves_til_win = [card.calculate_n_moves_until_win(numbers) for card in bingo_cards]

In [151]:
# Sorting ascendingly on n moves and taking the 1nd index we have our answer
sorted(moves_til_win, key=lambda tup: tup[0])[-1][1]

16836

### Day 5

#### Part 1

In [74]:
coordinates = read_input('day_5/input.txt')

# Coordinates is array of [[x1, y1], [x2, y2]] pairs
coordinates = np.array([[l.strip().split(',') for l in line.split('->')] for line in coordinates]).astype(int)

# Only keep coordinates where x1 == x2 or y1 == y2
hor_ver_coordinates = coordinates[(coordinates[:, 0, 0] == coordinates[:, 1, 0]) |
                                  (coordinates[:, 0, 1] == coordinates[:, 1, 1])]

In [75]:
def generate_empty_map(coordinates):
    # Using coordinates calculate max map size
    max_x = coordinates[:, :, 0].max()+1
    max_y = coordinates[:, :, 1].max()+1
    # Initialize empty map with all zeros
    return np.zeros(shape=(max_y, max_x))

In [76]:
map_ = generate_empty_map(hor_ver_coordinates)

In [77]:
for (x1, y1), (x2, y2) in hor_ver_coordinates:
    if x1 == x2 or y1 == y2:
        # Draw lines in the map by adding one
        # Using min and max we prevent slicing from high to low value, numpy
        # returns an empty array when we do that
        map_[min(y1, y2): max(y1, y2)+1, min(x1, x2): max(x1, x2)+1] += 1

In [78]:
# By counting all places where the value is higher than one we get our answer
len(np.where(map_ > 1)[0])

5280

#### Part 2

In [79]:
map_ = generate_empty_map(coordinates)

for (x1, y1), (x2, y2) in coordinates:
    # Horizontal/vertical line
    if x1 == x2 or y1 == y2:
        # Draw lines in the map by adding one
        # Using min and max we prevent slicing from high to low value, numpy
        # returns an empty array when we do that
        map_[min(y1, y2): max(y1, y2)+1, min(x1, x2): max(x1, x2)+1] += 1
    # Diagonal line
    else:
        # Cannot use the min/max trick for both x and y coordinates as this will result
        # in a flipped diagonal. We can simply fix this by using a negative stepsize, which
        # will result in a reversed range, and therefore no flipped lines
        x_stepsize = 1 if x2 > x1 else -1
        y_stepsize = 1 if y2 > y1 else -1

        # Because we also used to add 1 to the highest x and y, we need to substract
        # one from the lowest x and y if we have a reversed range, otherwise the lines
        # will be one too short
        x2 = x2 - 1 if x2 < x1 else x2 + 1
        y2 = y2 - 1 if y2 < y1 else y2 + 1
        for x, y in zip(range(x1, x2, x_stepsize), range(y1, y2, y_stepsize)):
            map_[y][x] += 1

In [80]:
len(np.where(map_ > 1)[0])

16716

### Day 6

#### Part 1

In [24]:
lanternfish_timers = read_input('day_6/input.txt')[0].split(',')
lanternfish_timers = np.array(lanternfish_timers).astype(int)

In [27]:
from tqdm import tqdm

n_days = 80

def lanternfish_model(original_population, n_days):
    """ Given an initial population and an amount of days, returns the
    amount of lanternfish after n_days days. """
    original_population = np.array([i for i in original_population])
    for day in range(n_days):
        # Subtract 1 from every timer
        original_population -= 1

        # Calculate amount of new lanternfishes by counting the amount of -1 timers
        zero_timers = np.where(original_population == -1)[0]

        if zero_timers.size > 0:
            # Add new lanternfish with timer 8 for every original timer that hits -1
            original_population = np.concatenate((original_population, np.full(len(zero_timers), 8)))

            # Reset -1 fishes to 6
            original_population[zero_timers] = 6
    
    return original_population.size

In [29]:
lanternfish_model(lanternfish_timers, 80)

354564

#### Part 2

In [43]:
# Above method grows exponentially so it doesnt work for large amount of days
# New method uses a list of counts where the index is the timer of the fishes
counts = [list(lanternfish_timers).count(i) for i in range(9)]

for _ in range(256):
    # Shift all items in count list to the left, the first element becomes
    # the last element (fishes with timer 8)
    counts = counts[1:] + [counts[0]]
    # Because the original timer 0 fishes dont die, we have to
    # add their count to index 6, because that is their timer after
    # multiplying
    counts[6] += counts[-1]

sum(counts)


1609058859115

### Day 7

#### Part 1

In [73]:
crab_positions = read_input("day_7/input.txt")[0].split(',')
crab_positions = np.array(crab_positions).astype(int)

In [74]:
def calculate_constant_fuel_cost(crab_pos, target):
    return sum(abs(crab_positions - target))

In [75]:
# Brute force as the data is not THAT big
costs = [(target, calculate_constant_fuel_cost(crab_positions, target)) for target in range(max(crab_positions))]
sorted(costs, key=lambda tup: tup[1])[0][1]

336120

#### Part 2

In [85]:

def calculate_linear_fuel_cost(crab_pos, target):
    """ Uses triangle numbers to calculate the costs """
    return sum((abs(crab_positions - target) * (abs(crab_positions - target) + 1)) / 2)


In [87]:
# Brute force still works fast enough
costs = [(target, calculate_linear_fuel_cost(crab_positions, target)) for target in range(max(crab_positions))]
sorted(costs, key=lambda tup: tup[1])[0][1]

96864235.0

### Day 8

#### Part 1

In [107]:
signals = read_input("day_8/input.txt")
signals = [line.split('|') for line in signals]

In [108]:
outputs = [s[1] for s in signals]

# Simply count the amount a token of length 2/3/4/7 occurs
sum([len(s) in [2, 3, 4, 7] for s in ' '.join(outputs).split(' ')])

488

#### Part 2

In [111]:
signals = [''.join(s) for s in signals]

Logic from https://imgur.com/a/LIS2zZr

- len(2) == 1
- len(4) == 4
- len(3) == 7
- len(7) == 8
- len(5) == [2, 3, 5]
    - contains(7) == 3
    - len(insersection(4) == 3) == 5
    - else 2
- len(6) == [0, 6, 9]
    - contains(4) == 9
    - contains(7) == 0
    - else 6

In [182]:
def token_to_charset(token):
    return set([c for c in token])

def charset_by_length(line, length):
    """ Used for getting set of the characters in codes for 1, 4, 7, and 8 """
    return set([[c for c in code] for code in line if len(code) == length][0])

def decode_line(line):
    # Dictionary foor looking up a number by length (for the numbers 1, 4, 7, 8)
    len_dict = {2: 1, 3: 7, 4: 4, 7: 8}

    # Dictionary with sets of characters for known numbers 1, 4, 7, and 8, using the
    # keys and values from len_dict
    char_dict = {k: charset_by_length(line, l) for l, k in len_dict.items()}

    decoded_tokens = []

    # Decoding using a for loop
    for token in line:
        charset = token_to_charset(token)
        # Simple lookup by length
        if len(charset) in [2, 3, 4, 7]:
            decoded_tokens.append(len_dict[len(charset)])
        # Length of 5 means 2, 3 or 5
        elif len(charset) == 5:
            # If the token contains all characters used by 7
            # the token is a three
            if char_dict[7].issubset(charset):
                decoded_tokens.append(3)
            # If the intersection of the characters is of size
            # 3 the token is a 5
            elif len(char_dict[4].intersection(charset)) == 3:
                decoded_tokens.append(5)
            # Only possible value is a 2
            else:
                decoded_tokens.append(2)
        # Length of 6 means 9, 0, or 6
        elif len(charset) == 6:
            # If the token contains all characters used by 4, the
            # token is a nine
            if char_dict[4].issubset(charset):
                decoded_tokens.append(9)
            # Same trick as above for the characters of 7, the token
            # is a 0
            elif char_dict[7].issubset(charset):
                decoded_tokens.append(0)
            # Final option a 6
            else:
                decoded_tokens.append(6)

    # Only return the final 4 values as integer, the rest is not needed for solution
    return int(''.join([str(i) for i in decoded_tokens[-4:]]))

In [184]:
sum([decode_line(line.split()) for line in signals])

1040429

### Day 9

#### Part 1

In [11]:
with open('day_9/input.txt', 'r') as f:
    heatmap = [[int(c) for c in line.strip()] for line in f.readlines()]
    
width = len(heatmap[0])
height = len(heatmap)

In [12]:
low_spot_values = []

# Simply march over every spot on the map and check if the spot is a low spot
for y in range(height):
    for x in range(width):
        curr_value = heatmap[y][x]
        # Hardcode the 4 points that we want to check
        #                  up        down      left      right
        coords_to_check = [[y-1, x], [y+1, x], [y, x-1], [y, x+1]]
        # By default assume the spot is a low point, we set this boolean to False
        # if we find a surrounding spot that is as low or lower as the current spot
        # we're looking at
        is_low_point = True
        for y_, x_ in coords_to_check:
            # Simply ignore index errors instead of manually checking for
            # edges and corners
            try:
                if heatmap[y_][x_] <= curr_value:
                    is_low_point = False
                    # Stop iterating as we already know the spot is not a low spot
                    break
            except IndexError:
                pass
            
        if is_low_point:
            low_spot_values.append(curr_value)
        

In [14]:
# Answer is the sum of all low spots + 1
sum(low_spot_values) + len(low_spot_values)

502

#### Part 2

### Day 10

In [45]:
with open('day_10/input.txt', 'r') as f:
    chunks = [line.strip() for line in f.readlines()]

In [54]:
opening_brackets = ['{', '[', '(', '<']
closing_brackets = ['}', ']', ')', '>']
opening_bracket_dict = {k: v for k, v in zip(opening_brackets, closing_brackets)}
closing_bracket_dict = {k: v for k, v in zip(closing_brackets, opening_brackets)}

In [52]:
def parse_corrupted_line(line):
    to_close = []
    
    for char in line:
        # For every opening bracket we must find a closing bracket, otherwise
        # the line is currupt
        if char in opening_brackets:
            to_close.append(char)
        # If we find a closing bracket it must be for the last found
        # opening bracket, if this is not true the line is corrupt
        # and we return the corrupting character
        elif char in closing_brackets:
            if to_close[-1] == closing_bracket_dict[char]:
                to_close.pop(-1)
            else:
                return char


illegal_char_cost = {k: v for k, v in zip([')', ']', '}', '>'], [3, 57, 1197, 25137])}
            
illegal_chars = [parse_corrupted_line(line) for line in chunks]

In [53]:
sum([illegal_char_cost[c] for c in illegal_chars if c is not None])

362271

#### Part 2

In [64]:
def complete_line(line):
    to_close = []
    for char in line:
        # Same logic as part 1, without stopping if we find a corrupt character (because
        # we can assume these lines are not ccorrupted)
        if char in opening_brackets:
            to_close.append(char)
        elif char in closing_brackets:
            if to_close[-1] == closing_bracket_dict[char]:
                to_close.pop(-1)
                
    # Output is the closing brackets for the brackets in to_close, reversed
    return [opening_bracket_dict[c] for c in to_close[::-1]]

[')', '}', '>', ']', '}', ')']

In [71]:
def score_line(line):
    score = 0
    for i in line:
        score *= 5
        score += i
    return score

illegal_char_cost = {k: v for k, v in zip([')', ']', '}', '>'], [1, 2, 3, 4])}

completions = [complete_line(line) for line in chunks if parse_corrupted_line(line) is None]
completion_scores = [score_line([illegal_char_cost[c] for c in line]) for line in completions]

In [73]:
sorted(completion_scores)[len(completion_scores) //2]

1698395182

### Day 11

#### Part 1

In [121]:
with open("day_11/input.txt", 'r') as f:
    octopi = np.array([[int(c) for c in line.strip()] for line in f.readlines()])
    
height = len(octopi)
width = len(octopi[0])

# Pad array with a large negative value so we dont have to worry about
# index errors around the edges
octopi = np.pad(octopi, 1, 'constant', constant_values=-10000)

In [119]:
total_flashes = 0

for _ in range(100):
    # Default add 1 to every value at the start of step
    octopi += 1
    
    # List to keep track of all flashes octopi, they can only flash once
    flashed = []
    # List of tuple y, x coordinates where an octopi has flashed
    flashes = list(zip(*np.where(octopi > 9)))
    # While we have flashing octopi
    while flashes:
        # Save flashed octopi to the list so that they wont flash again
        flashed.extend(flashes)
        # Add 1 to every octopi around a flashing octupus
        for y_, x_ in flashes:
            octopi[y_-1:y_+2, x_-1: x_+2] += 1

        # Find new octopi that are going to flash, ignoring the ones that have already flashed
        flashes = [tup for tup in list(zip(*np.where(octopi > 9))) if tup not in flashed]
    # Reset counter for all flashed octopi
    octopi[octopi > 9] = 0
    # Keep track of amount of flashes for the answer
    total_flashes += len(flashed)
    
total_flashes

1594

#### Part 2

In [122]:
for step in range(10000):
    # Default add 1 to every value at the start of step
    octopi += 1
    
    # List to keep track of all flashes octopi, they can only flash once
    flashed = []
    # List of tuple y, x coordinates where an octopi has flashed
    flashes = list(zip(*np.where(octopi > 9)))
    # While we have flashing octopi
    while flashes:
        # Save flashed octopi to the list so that they wont flash again
        flashed.extend(flashes)
        # Add 1 to every octopi around a flashing octupus
        for y_, x_ in flashes:
            octopi[y_-1:y_+2, x_-1: x_+2] += 1

        # Find new octopi that are going to flash, ignoring the ones that have already flashed
        flashes = [tup for tup in list(zip(*np.where(octopi > 9))) if tup not in flashed]
    # Reset counter for all flashed octopi
    octopi[octopi > 9] = 0
    
    if octopi[1: height+1, 1: width+1].sum() == 0:
        print(step + 1)
        break
    

437


### Day 12

#### Part 1

In [143]:
with open("day_12/input.txt", 'r') as f:
    edges = [line.strip().split('-') for line in f.readlines()]

In [144]:
connections = {}
for from_, to in edges:
    if from_ not in connections:
        connections[from_] = []
    if to not in connections:
        connections[to] = []
    connections[from_].append(to)
    connections[to].append(from_)

In [145]:
def paths(curr, seen):
    if curr == 'end':
        return 1
    # Can only visit once
    if curr.islower() and curr in seen:
        return 0
    seen = seen | {curr}
    out = 0
    for cave in connections[curr]:
        out += paths(cave, seen)
    return out

paths("start", set())

4775

#### Part 2

In [148]:
def paths(curr, seen, dup):
    if curr == 'end':
        return 1
    if curr == "start" and seen:
        return 0
    if curr.islower() and curr in seen:
        if dup is None:
            dup = curr
        else:
            return 0
    seen = seen | {curr}
    out = 0
    for cave in connections[curr]:
        out += paths(cave, seen, dup)
    return out

paths("start", set(), None)

152480

### Day 13

#### Part 1

In [327]:
with open("day_13/input.txt", 'r') as f:
    coordinates, instructions = f.read().split('\n\n')
coordinates = np.array([[int(c) for c in line.split(',')] for line in coordinates.split('\n')])
instructions = [line.split('=') for line in instructions.split('\n')][:-1]
instructions = [[l[0][-1], int(l[1])] for l in instructions]

In [328]:
def fold_paper(paper, axis, value):
    if axis == 'x':
        paper = paper[:, :value] + np.flip(paper[:, value + 1:], axis=1)
    else:
        paper = paper[:value, :] + np.flip(paper[value + 1:, :], axis=0)
    paper = np.clip(paper, 0, 1)
    
    # Have paper always have odd dimensions, otherwise flip doesnt work because of
    # different array shapes
    if len(paper) % 2 == 0:
        paper = np.vstack([paper, np.zeros(len(paper[0]))])
        
    if len(paper[0]) % 2 == 0:
        paper = np.hstack([paper, np.zeros(len(paper)).reshape(-1, 1)])
    
    return paper

In [329]:
paper = np.zeros(shape=(coordinates[:, 1].max() + 1, coordinates[:, 0].max() + 1))
paper[coordinates[:, 1], coordinates[:, 0]] = 1

for axis, value in instructions[:1]:
    paper = fold_paper(paper, axis, value)
    

paper.sum()

755.0

#### Part 2

In [330]:
paper = np.zeros(shape=(coordinates[:, 1].max() + 1, coordinates[:, 0].max() + 1))
paper[coordinates[:, 1], coordinates[:, 0]] = 1


for axis, value in instructions:
    paper = fold_paper(paper, axis, value)

In [331]:
print('\n'.join([''.join(row) for row in paper.astype(int).astype(str).tolist()]).replace('1', '#').replace('0', ' '))

###  #    #  #   ## ###  ###   ##   ##   
#  # #    # #     # #  # #  # #  # #  #  
###  #    ##      # #  # ###  #  # #     
#  # #    # #     # ###  #  # #### # ##  
#  # #    # #  #  # # #  #  # #  # #  #  
###  #### #  #  ##  #  # ###  #  #  ###  
                                         


### Day 14

#### Part 1

In [273]:
from collections import Counter

with open("day_14/input.txt", 'r') as f:
    template, rules = f.read().split("\n\n")
rules = [line.split(' -> ') for line in rules.split('\n')][:-1]
rules = {line[0]: line[1] for line in rules}

In [274]:
for _ in range(10):
    new = ""

    for c1, c2 in zip(template[:-1], template[1:]):
        new += c1 + rules[c1+c2]
    new += template[-1]
    template = new

In [275]:
count = Counter(template).most_common(99)
count[0][1] - count[-1][1]

2712

#### Part 2

In [294]:
from tqdm import tqdm
from collections import defaultdict

with open("day_14/input.txt", 'r') as f:
    template, _ = f.read().split("\n\n")

pair_dict = defaultdict(int)
char_count = Counter(template)

# Count pairs instead of keeping whole string in memory
for c1, c2 in zip(template[:-1], template[1:]):
    pair_dict[c1+c2] += 1

    
for _ in range(40):
    for pair, count in pair_dict.copy().items():
        # Because we're splitting the pair in two it should be removed from the pair count
        pair_dict[pair] -= count
        # Add count of original pair to both new pairs that happen when inserting new character
        pair_dict[pair[0]+rules[pair]] += count
        pair_dict[rules[pair]+pair[1]] += count
        # Also update the count of the new characters we added to the 'string'
        char_count[rules[pair]] += count

In [295]:
char_count = sorted(char_count.items(), key=lambda tup: tup[1])
char_count[-1][1] - char_count[0][1]

8336623059567

### Day 15

#### Part 1

In [523]:
import networkx as nx

with open('day_15/input.txt', 'r') as f:
    map_ = [[int(c) for c in line.strip()] for line in f.readlines()]

In [524]:
def network_from_map(map_):
    """ Simply transform the map into a directed networkx graph so we can use the builtin
    shortest path function to calculate the sum of the safest path. """
    width = len(map_[0])
    height = len(map_)
    
    edge_list = []

    for y in range(height):
        for x in range(width):
            curr = f"{y}-{x}"
                        # Top     # Bottom   # Left    # Right
            to_visit = [(y-1, x), (y+1, x), (y, x-1), (y, x+1)]
            for y_, x_ in to_visit:
                if y_ >= 0 and x_ >= 0 and y_ < height and x_ < width:
                    edge_list.append((curr, f"{y_}-{x_}", map_[y_][x_]))        
    
    g = nx.DiGraph()
    g.add_weighted_edges_from(edge_list)
    
    return g

In [525]:
g = network_from_map(map_)
shortest_path_coords = [coords.split('-') for coords in nx.shortest_path(g, '0-0', f"{height-1}-{width-1}", weight='weight')]


sum([map_[int(y)][int(x)] for y, x in shortest_path_coords[1:]])

386

#### Part 2

In [527]:
map_ = np.array(map_)

In [528]:
large_map = [[[] for _ in range(5)] for _ in range(5)]
large_map[0][0] = map_

In [529]:
def transform_map(map_):
    new_map = map_ + 1
    new_map[new_map > 9] = 1
    return new_map

In [530]:
for y in range(5):
    if y == 0:
        pass
    else:
        large_map[y][0] = transform_map(large_map[y-1][0])
    for x in range(1, 5):
        large_map[y][x] = transform_map(large_map[y][x-1])
        

In [531]:
# Concat all rows and finally stack the rows on top of eachother to get the map
map_ = np.vstack([np.hstack(row) for row in large_map])
map_.shape

(500, 500)

In [532]:
g = network_from_map(map_)
shortest_path_coords = [coords.split('-') for coords in
                        nx.shortest_path(g, '0-0', f"{map_.shape[0]-1}-{map_.shape[1]-1}", weight='weight')]


sum([map_[int(y)][int(x)] for y, x in shortest_path_coords[1:]])

2806

### Day 16

#### Part 1

In [178]:
with open("day_16/input.txt", 'r') as f:
    hexcode = f.read().strip()

In [179]:
decoding_num = {str(i): bin(i)[2:].zfill(4) for i in range(10)}
decoding_char = {tup[0]: bin(tup[1])[2:].zfill(4) for tup in zip("ABCDEF", [i for i in range(10, 16)])}
decoding = {**decoding_num, **decoding_char}

In [180]:
decoded = ''.join([decoding[c] for c in hexcode])

In [182]:
version_sum = 0


def parse(data):
    global version_sum
    # First 3 bits are version
    version = int(data[:3], 2)
    version_sum += version
    # We parsed version so we can remove it from the data
    data = data[3:]
    
    # Same trick as above for getting version
    type_ = int(data[:3], 2)
    data = data[3:]
    
    # Part 1 doesnt need anything done for type four as long as we saved the version
    if type_ == 4:
        while True:
            keep_going = data[0]
            data = data[5:]
            if keep_going == '0':
                break
    else:
        length_type_id = data[0]
        data = data[1:]
        # Recursively parse the subparts using the defined length type id
        if length_type_id == '0':
            subp_length = int(data[:15], 2)
            data = data[15:]
            subp = data[:subp_length]
            while subp:
                subp = parse(subp)
            data = data[subp_length:]
        else:
            n_subp = int(data[:11], 2)
            data = data[11:]
            for _ in range(n_subp):
                data = parse(data)
        
    return data


In [183]:
_ = parse(decoded)
version_sum

963

#### Part 2

In [184]:
# TODO

### Day 17

#### Part 1

In [1]:
import re

with open("day_17/input.txt", 'r') as f:
    target_area = f.read().strip()
    
xl, xr, yb, yt = [int(i) for i in re.findall(r'-?\d+', target_area)]
target_area, xl, xr, yb, yt

('target area: x=150..171, y=-129..-70', 150, 171, -129, -70)

In [10]:
def traj(xv, yv):
    max_ypos = 0
    xpos, ypos = 0, 0
    # As long as we are not overshooting keep going
    while xpos <= xr and ypos >= yb:
        xpos += xv
        ypos += yv
        xv -= 1 if xv > 0 else -1 if xv < 0 else 0
        yv -= 1
        
        if ypos > max_ypos:
            max_ypos = ypos
        # Check if we are in target zone
        if xpos >= xl and xpos <= xr and ypos >= yb and ypos <= yt:
            return True, max_ypos
        
    return False, 0

In [3]:
# x pos converges after xv steps, so we first brute-force the valid initial x velocities

In [4]:
def xpos_convergence(n):
    """ Basically triangle number. """
    return sum([i for i in range(1, n+1)])

In [5]:
xv = 0
valid_xvs = []
while True:
    # Keep track of amount of valid x velocities; this is used for stopping the loop
    # when we are overshooting
    curr_len = len(valid_xvs)
    final_xpos = xpos_convergence(xv)
    # If convergence is in target we save the x velocity
    if final_xpos >= xl and final_xpos <= xr:
        valid_xvs.append(xv)
    xv += 1
    
    # If we already added to list and the length hasnt changed we are overshooting the target
    # and we can stop finding x velocities
    if len(valid_xvs) and len(valid_xvs) == curr_len:
        break

valid_xvs

[17, 18]

In [6]:
valid_shots = []

for xv in valid_xvs:
    for yv in range(1000):
        valid_traj, max_ypos = traj(xv, yv)
        if valid_traj:
            valid_shots.append((xv, yv, max_ypos))
            

In [7]:
sorted(valid_shots, key=lambda tup: tup[2])[-1][2]

8256

#### Part 2

In [11]:
from tqdm import tqdm

# Rather wait a few seconds brute forcing instead of using math to find all succesful velocities

good_init_velocities = 0

for xv in tqdm(range(1000)):
    for yv in range(-1000, 1000):
        valid_traj, _ = traj(xv, yv)
        if valid_traj:
            good_init_velocities += 1

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 134.44it/s]


In [12]:
good_init_velocities

2326

### Day 18

### Day 19

### Day 20

#### Part 1

In [185]:
from tqdm import tqdm

def calc_output_pixel(arr):
    bytes_ = ''.join(arr.flatten().astype(str).tolist())
    idx = int(bytes_, 2)
    return enhancement_string[idx]


def print_img(img):
    print('\n'.join([''.join(line) for line in img.astype(str)]).replace('0', '.').replace('1', '#'))

In [191]:
with open("day_20/input.txt") as f:
    enhancement_string, img = f.read().split("\n\n")
        
img = img.replace('.', '0').replace('#', '1').strip().split('\n')
img = np.array([[int(c) for c in line] for line in img])

enhancement_string = enhancement_string.replace('.', '0').replace('#', '1')

In [192]:
def expand_image(img, n_steps):
    # Pad image by 5 to keep some buffer for expanding
    img = np.pad(img, 5, 'constant', constant_values=0)
    
    for _ in tqdm(range(n_steps)):
        # Pad 'infinite' image with 2 rows/columns containing the values of the edge
        # This handles potential growth of the image while keeping an extra buffer that we will
        # trim off later, as our for loop wont update the edges
        img = np.pad(img, 2, 'edge')

        # Place to store new values
        new_img = img.copy()

        # Iterate over whole image except the edges to prevent index errors
        for y in range(1, len(img)-1):
            for x in range(1, len(img[0]) - 1):
                new_img[y, x] = calc_output_pixel(img[y-1:y+2, x-1:x+2])

        # Trim away the edges we did not iterate over
        img = new_img[1:-1, 1:-1].copy()
    
    return img

In [193]:
expand_image(img, 2).sum()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  9.25it/s]


5622

#### Part 2

In [194]:
# Somehow code was fast enough to do part 2 aswell
expand_image(img, 50).sum()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  3.04it/s]


20395

### Day 21

#### Part 1

In [217]:
with open("day_21/input.txt") as f:
    p1, p2 = [int(line[-1]) for line in f.read().strip().split("\n")]
    
p1, p2

(6, 4)

In [224]:
class DeterministicDice:
    def __init__(self):
        self.prev_roll = 0
        self.n_rolls = 0
        
    def roll(self):
        self.n_rolls += 1
        
        if self.prev_roll == 100:
            self.prev_roll = 1
        else:
            self.prev_roll += 1
        return self.prev_roll
    
    def sum_of_n_rolls(self, n=3):
        return sum([self.roll() for _ in range(n)])
    
    
class Board:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2
        self.p1_score = 0
        self.p2_score = 0
        self.winner = None
        
    def move(self, player, n):
        if player == 1:
            self.p1 = (self.p1 + n) % 10 if self.p1 + n != 10 else 10
            self.p1_score += self.p1
        else:
            self.p2 = (self.p2 + n) % 10 if self.p2 + n != 10 else 10
            self.p2_score += self.p2
            
    def check_for_winner(self):
        if self.p1_score >= 1000:
            self.winner = 1
            return True
        elif self.p2_score >= 1000:
            self.winner = 2
            return True
        else:
            return False
        
    
dice = DeterministicDice()

In [225]:
dice = DeterministicDice()
board = Board(p1, p2)

player = 0

while board.winner is None:
    roll = dice.sum_of_n_rolls(3)
    board.move(player + 1, roll)
    board.check_for_winner()
    player = abs(player - 1)

In [228]:
if board.winner == 1:
    losing_score = board.p2_score
else:
    losing_score = board.p1_score
    
losing_score * dice.n_rolls

920580